# Introduction

We are proud to share that we won QuEra Computing’s challenge at the ETH Zurich Quantum Hackathon. The event spanned three intense and rewarding days. After the opening keynote, we were introduced to the five available challenges and had the opportunity to briefly visit ETH Zurich’s cutting-edge research facilities. Among the proposed topics, we were particularly drawn to QuEra’s challenge, which focused on optimizing quantum circuits for their neutral-atom quantum computing hardware — a problem that resonated strongly with our background in Computer Engineering and High Performance Computing.

On the second day, we met Pedro and John from the QuEra Computing team, who presented the technical details of the task and provided valuable guidance throughout the competition. Once the challenge was fully explained, the 25-hour countdown began—and so did our focused effort.

As this was our first hackathon experience, we found it to be deeply educational, technically demanding, and ultimately very fulfilling. What set our team apart was our distinctive approach, different from that of most other participants, shaped by our perspective as computer engineers.

In this post, we share our experience and reflect on the process that led us to our final solution.


# Our Approach

During the initial phase of the challenge, we dedicated our time to understand the provided resources: the detailed challenge description document, associated academic papers, and tutorial notebooks showcasing QuEra’s software tools. This foundational phase was crucial for building a strong conceptual framework.

In subsequent discussions with John and Pedro, we realized there was an opportunity to delve deeper into their software infrastructure. Using built-in debugging features, we explored the internal architecture of their compiler, which allowed us to gain insights and  design our own optimization passes.

As we entered the second half of the challenge, we  split our efforts into two parallel tasks to maximize efficiency during the intensive overnight session:

- One subgroup focused on developing custom optimization passes.
- The other subgroup worked on manually analyzing and parallelizing circuits.

To objectively evaluate our results, we established a set of clear metrics, prioritized as follows:
1. Fidelity – ensuring that the optimized circuits maintained equivalence to the originals.
2. Total CZ gate count – more relevant than U gates due to the higher cost of CZ gates.
3. Total U gate count – aiming to minimize overall gate usage.
4. Parallel CZ gate count.
5. Parallel U gate count.

Ideal optimization corresponds to fidelity close to 1 and minimized gate counts, with a high degree of parallelization.


## Passes

Each pass was specifically crafted to address bottlenecks identified during initial profiling of the circuits. 
These passes included:

- Redundant Gate Elimination: removes 2pi gates that are introduced during the internal Rewrite
- U Gate Merge Pass: combined adjacent single-qubit gates to reduce overhead

We also used native passes from the Bloqade library:

- Rydberg Gate Set Rewrite: this pass translates all gates in the given circuits into CZ and U gates (the hardware gate set)
- UOpToParallel: this pass identifies and rewrites gates that can be parallelized

The following code snippets highlight the effect of these passes:    

## Manual Optimization

The main point was parallelizing CZ gates thanks to QuEra's unique architecture, but for all the given circuits we adpoted the same approach:

1. Initial inspection to analyze gate dependencies and overall structure 
2. Transformation into an equivalent but more parallel-friendly circuit layout
3. Strategic reordering of gates to maximize parallel execution
4. Careful validation of results by verifying the fidelity between the improved and the original circuits

To better explain this technique, we present here as an example the job done on circuit number 4 (surface code encoding circuit). 
This is how the initial circuit was like:

In [ ]:
# Print 4

We can see that there are 5 useless gates, i.e. the initial CX gates whose control qubits are |0>. 